In [1]:
for number in range(-2, 3):
    try:
        print(10 / number)
    except ZeroDivisionError:
        continue    # continue only possible if try-except is inside the loop

-5.0
-10.0
10.0
5.0


## Generator
Use generator to provide functions and collect results. Seems to be rather a game instead of a useful technique.

In [2]:
from typing import Any, Generator, Callable, Iterable

ActionReturn = Any
Action = Callable[[], ActionReturn]

def consume_exceptions(
        actions: Generator[Action, ActionReturn, None]) -> None:
    """Execute actions and send back results using the given generator.
    
    Args:
        actions: generator yeilding argumentless functions and accepting
            results (including caught exceptions)
    """
    action = next(actions)
    while True:
        try:
            result = action()
        except Exception as exception:
            result = exception  # or send the exception as result
        try:
            action = actions.send(result)
        except StopIteration:   # generator used up
            return

In [8]:
def do_smth0():
    print('Entering do_smth0.')
    return 'Value from do_smth0'

def do_smth1():
    print('Entering do_smth1.')
    return 1 / 0

def do_smth2():
    print('Entering do_smth2.')
    return 'Value from do_smth2'

def do_smth3():
    print('Entering do_smth3.')
    return 3

In [14]:
def do_many_things() -> None:
    """Provide an example of preparation of generator.
    
    The generator provides actions (functions) and uses the results.
    """
    print(repr((yield do_smth0)))
    print(repr((yield do_smth1)))
    print(repr((yield do_smth2)) + ' do something with the result')
    print(2 * (yield do_smth3))
    print(1 + (yield lambda: 5 * 5))
    
consume_exceptions(do_many_things())

Entering do_smth0.
'Value from do_smth0'
Entering do_smth1.
ZeroDivisionError('division by zero')
Entering do_smth2.
'Value from do_smth2' do something with the result
Entering do_smth3.
6
26


In [10]:
def do_many_things_generic(
        functions: Iterable[Action],
        results: list[ActionReturn | BaseException]) ->None:
    for function in functions:
        print(f'DBG: giving: {function.__name__}')
        results.append((yield function))  # yield must be inside parentheses to make an expression
        print(f'DBG:  - collected: {results[-1]!r}')
        print()

results = []
consume_exceptions(do_many_things_generic(
        (do_smth0, do_smth1, do_smth2, do_smth3), results))
print(f'{results = }')

DBG: giving: do_smth0
Entering do_smth0.
DBG:  - collected: 'Value from do_smth0'

DBG: giving: do_smth1
Entering do_smth1.
DBG:  - collected: ZeroDivisionError('division by zero')

DBG: giving: do_smth2
Entering do_smth2.
DBG:  - collected: 'Value from do_smth2'

DBG: giving: do_smth3
Entering do_smth3.
DBG:  - collected: 3

results = ['Value from do_smth0', ZeroDivisionError('division by zero'), 'Value from do_smth2', 3]
